In [2]:
import torch # finally get to use pytorch again
import torch.nn as nn
import torch.nn.functional as F
from collections import deque

import gymnasium as gym
import numpy as np

In [3]:
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset()
action = env.action_space.sample()

def generate_episode(n_ep, env):
    for _ in range(n_ep):
        observation, info = env.reset()

        while True:
            action = env.action_space.sample()
            observation, reward, terminated, truncated, info = env.step(action)
            
            if terminated or truncated:
                break
env.close()

In [4]:
class Q_network(nn.Module):
    def __init__(self, in_dim, out_dim) -> None:
        super().__init__()
        # note to self: nn.Linear() represents the transformation, not the matrices themselves.
        self.il = nn.Linear(in_dim, 50)
        self.ol = nn.Linear(50, out_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = F.relu(self.il(x))
        return F.relu(self.ol(x))

# shamelessly copied off of 
class Replay_Memory():
    def __init__(self, cap):
        self.memory = deque([], maxlen = cap)

    def push(self, *args):
        self.memory.append(*args)
        
    def sample(self, batch_size):
        return np.random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
def epsilon_greedy(Q_net, S, epsilon):
    r = torch.random.random()
    if r > epsilon:
        return torch.argmax(Q_net.forward(S))
    else:
        return torch.random.choice([0, 1, 2, 3])

def DQL(_env, n_ep):
    state_dim = _env.observation_space.shape[0]
    action_dim = _env.action_space.n
    
    buffer_cap = 100000
    replay_buffer = Replay_Memory(buffer_cap)
    
    behaviour_net = Q_network(state_dim, action_dim)
    target_net = Q_network(state_dim, action_dim)
    optimizer = torch.optim.Adam(behaviour_net.parameters(), lr=0.01)
    
    decay_min = 0.01
    decay_const = 0.01
    
    successes = 0
    _gamma = 0.99
    # _alpha = 0.5

    for i in range(n_ep):
        epsilon = decay_min + (1 - decay_min) * np.exp(-decay_const * i)
        observation, info = _env.reset()
        S = observation
        
        if (i % 500) == 0:
            target_net.load_state_dict(behaviour_net.state_dict())
            
        while not done:
            A = epsilon_greedy(behaviour_net, S, epsilon)
            S_prime, R, terminated, truncated, info = _env.step(A)
            done = terminated or truncated
            replay_buffer.push(S, A, R, S_prime, done)

            S = S_prime
            
            if len(replay_buffer) > 64:
                # sample from replay buffer
                state_batch, action_batch, reward_batch, next_state_batch, done_batch = replay_buffer.sample(batch_size)
                # feed these values to Q-network, get the Q-values
                q_values = behaviour_net(state_batch)
                # do magic shit to get Q-value for the action taken
                q_values_for_actions = q_values.gather(1, action_batch.unsqueeze(1)).squeeze(1)

                # get max q-value 
                with torch.no_grad():
                    next_q_values = target_net(next_state_batch).max(1)[0]

                # calc target q-value estimation
                
            loss = (Q_net.forward(S) - target_q) ** 2
            loss.backward()
            optimizer.step()
            
            
            if terminated or truncated:
                if R == 1:
                    successes += 1
                break
                
    print("Number of successes:", successes)
    return behaviour_net